# BUILDING CUSTOMER DIMENSION TABLE

# Import libraries 

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import trim, col
from pyspark.sql.types import StringType

# Mapping/rules 

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_key",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "create_date",
}

# Reading from Bronze

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

# Data Transformations



## Renaming the columns

In [0]:
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

## Trim whitespace for every string column

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name,trim(col(field.name)))

## Normalization/Standardizes columns into friendly values

In [0]:
df = (
    df
    .withColumn(
        "marital_status",
        F.when(F.upper(F.col("marital_status")) == "S", "Single")
        .when(F.upper(F.col("marital_status")) == "M", "Married")
        .otherwise(None)
    )

    .withColumn(
        "gender",
        F.when(F.upper(F.col("gender")) == "M", "Male")
        .when(F.upper(F.col("gender")) == "F", "Female")
        .otherwise(None)
    )
)

## Check customer_key for nulls

In [0]:
# rows where customer_key is missing
missing_key_count = df.filter(
    F.col("customer_key").isNull() | (F.trim(F.col("customer_key")) == "")
).count()

print(f'Total of missing keys is  {missing_key_count}.')


## Check customer_key for duplicates 

In [0]:
duplicate_key_groups = (
    df.filter(F.col("customer_key").isNotNull() & (F.trim(F.col("customer_key")) != ""))
      .groupBy("customer_key")
      .count()
      .filter(F.col("count") > 1)
      .count()
)
print(f'There are {duplicate_key_groups} duplicate keys.')

## List the keys and how many times they repeat

In [0]:
dups = (
    df.filter(F.col("customer_key").isNotNull() & (F.trim(F.col("customer_key")) != ""))
      .groupBy("customer_key")
      .count()
      .filter(F.col("count") > 1)
      .orderBy(F.col("count").desc())
)

display(dups)


In [0]:
key = dups.first()["customer_key"]
display(df.filter(F.col("customer_key") == key))


In [0]:
df.display()

In [0]:
%sql
select marital_status, count(*)
from workspace.silver.crm_customers
group by marital_status;


# Write Into Silver Table

In [0]:
(
    df.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("silver.crm_customers")
)

# Check data

In [0]:
%sql
select * from workspace.silver.crm_customers